In [41]:
import pandas as pd
import numpy as np

data =  open('links.txt').read().splitlines()
PATH = 'data/'
data



['turnstile_160507.txt',
 'turnstile_160514.txt',
 'turnstile_160521.txt',
 'turnstile_160528.txt',
 'turnstile_160604.txt',
 'turnstile_160611.txt',
 'turnstile_160618.txt',
 'turnstile_160625.txt',
 'turnstile_160702.txt']

In [42]:
dfs = [pd.read_csv(PATH + path) for path in data]
df = pd.concat(dfs, ignore_index=True)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1747984 entries, 0 to 1747983
Data columns (total 11 columns):
C/A                                                                     object
UNIT                                                                    object
SCP                                                                     object
STATION                                                                 object
LINENAME                                                                object
DIVISION                                                                object
DATE                                                                    object
TIME                                                                    object
DESC                                                                    object
ENTRIES                                                                 int64
EXITS                                                                   int64
dtypes: int64(2), ob

In [44]:
df.columns = df.columns.str.strip()

df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [45]:
df['DATETIME'] = pd.to_datetime(df.DATE + " " + df.TIME, format='%m/%d/%Y %H:%M:%S')
df.head(5)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
0,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,00:00:00,REGULAR,5639941,1909983,2016-04-30 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,04:00:00,REGULAR,5639991,1909993,2016-04-30 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,08:00:00,REGULAR,5640014,1910024,2016-04-30 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,12:00:00,REGULAR,5640158,1910134,2016-04-30 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,16:00:00,REGULAR,5640454,1910197,2016-04-30 16:00:00


In [46]:
# Make sure there are no duplicate entries
df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATETIME"], inplace=True)

In [47]:
df[["PREV_DATETIME", "PREV_ENTRIES", "PREV_EXITS"]] = (df.groupby(["C/A", "UNIT", "SCP", "STATION"])["DATETIME", "ENTRIES", "EXITS"]
                                            .transform(lambda grp: grp.shift(1)))
df.head(5)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,PREV_DATETIME,PREV_ENTRIES,PREV_EXITS
0,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,00:00:00,REGULAR,5639941,1909983,2016-04-30 00:00:00,NaT,NaN,NaN
1,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,04:00:00,REGULAR,5639991,1909993,2016-04-30 04:00:00,2016-04-30 00:00:00,5639941.0,1909983.0
2,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,08:00:00,REGULAR,5640014,1910024,2016-04-30 08:00:00,2016-04-30 04:00:00,5639991.0,1909993.0
3,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,12:00:00,REGULAR,5640158,1910134,2016-04-30 12:00:00,2016-04-30 08:00:00,5640014.0,1910024.0
4,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,16:00:00,REGULAR,5640454,1910197,2016-04-30 16:00:00,2016-04-30 12:00:00,5640158.0,1910134.0


In [48]:
df.dropna(subset=["PREV_DATETIME"], axis=0, inplace=True)

In [49]:
df['TIME_INTERVAL'] = df['DATETIME'] - df['PREV_DATETIME']
df = df.drop(df[(df['TIME_INTERVAL'] > '05:00:00') & (df['TIME_INTERVAL'] < '03:00:00')].index)
df.head(5)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,PREV_DATETIME,PREV_ENTRIES,PREV_EXITS,TIME_INTERVAL
1,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,04:00:00,REGULAR,5639991,1909993,2016-04-30 04:00:00,2016-04-30 00:00:00,5639941.0,1909983.0,04:00:00
2,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,08:00:00,REGULAR,5640014,1910024,2016-04-30 08:00:00,2016-04-30 04:00:00,5639991.0,1909993.0,04:00:00
3,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,12:00:00,REGULAR,5640158,1910134,2016-04-30 12:00:00,2016-04-30 08:00:00,5640014.0,1910024.0,04:00:00
4,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,16:00:00,REGULAR,5640454,1910197,2016-04-30 16:00:00,2016-04-30 12:00:00,5640158.0,1910134.0,04:00:00
5,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,20:00:00,REGULAR,5640802,1910254,2016-04-30 20:00:00,2016-04-30 16:00:00,5640454.0,1910197.0,04:00:00


In [50]:
# modify entry counts that are negative and remove the ones with outrageous values
df['ENTRY_COUNT'] = df.ENTRIES - df.PREV_ENTRIES
df.ix[df.ENTRY_COUNT < 0, 'ENTRY_COUNT'] =  - df['ENTRY_COUNT']
df = df.drop(df[df.ENTRY_COUNT > 1000000].index)

In [52]:
# modify exit counts that are negative and remove the ones with outrageous values
df['EXIT_COUNT'] = df.EXITS - df.PREV_EXITS
df.ix[df.EXIT_COUNT < 0, 'EXIT_COUNT'] =  - df['EXIT_COUNT']
df = df.drop(df[df.EXIT_COUNT > 1000000].index)
df.head(5)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,PREV_DATETIME,PREV_ENTRIES,PREV_EXITS,TIME_INTERVAL,ENTRY_COUNT,EXIT_COUNT
1,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,04:00:00,REGULAR,5639991,1909993,2016-04-30 04:00:00,2016-04-30 00:00:00,5639941.0,1909983.0,04:00:00,50.0,10.0
2,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,08:00:00,REGULAR,5640014,1910024,2016-04-30 08:00:00,2016-04-30 04:00:00,5639991.0,1909993.0,04:00:00,23.0,31.0
3,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,12:00:00,REGULAR,5640158,1910134,2016-04-30 12:00:00,2016-04-30 08:00:00,5640014.0,1910024.0,04:00:00,144.0,110.0
4,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,16:00:00,REGULAR,5640454,1910197,2016-04-30 16:00:00,2016-04-30 12:00:00,5640158.0,1910134.0,04:00:00,296.0,63.0
5,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,20:00:00,REGULAR,5640802,1910254,2016-04-30 20:00:00,2016-04-30 16:00:00,5640454.0,1910197.0,04:00:00,348.0,57.0


In [53]:
df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%d/%Y')
df['WEEKDAY'] = df['DATE'].dt.dayofweek
df['TIME'] = pd.to_datetime(df['TIME'], format='%H:%M:%S')
df['HOUR'] = df['TIME'].dt.hour
df.head(5)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,PREV_DATETIME,PREV_ENTRIES,PREV_EXITS,TIME_INTERVAL,ENTRY_COUNT,EXIT_COUNT,WEEKDAY,HOUR
1,A002,R051,02-00-00,59 ST,NQR456,BMT,2016-04-30,1900-01-01 04:00:00,REGULAR,5639991,1909993,2016-04-30 04:00:00,2016-04-30 00:00:00,5639941.0,1909983.0,04:00:00,50.0,10.0,5,4
2,A002,R051,02-00-00,59 ST,NQR456,BMT,2016-04-30,1900-01-01 08:00:00,REGULAR,5640014,1910024,2016-04-30 08:00:00,2016-04-30 04:00:00,5639991.0,1909993.0,04:00:00,23.0,31.0,5,8
3,A002,R051,02-00-00,59 ST,NQR456,BMT,2016-04-30,1900-01-01 12:00:00,REGULAR,5640158,1910134,2016-04-30 12:00:00,2016-04-30 08:00:00,5640014.0,1910024.0,04:00:00,144.0,110.0,5,12
4,A002,R051,02-00-00,59 ST,NQR456,BMT,2016-04-30,1900-01-01 16:00:00,REGULAR,5640454,1910197,2016-04-30 16:00:00,2016-04-30 12:00:00,5640158.0,1910134.0,04:00:00,296.0,63.0,5,16
5,A002,R051,02-00-00,59 ST,NQR456,BMT,2016-04-30,1900-01-01 20:00:00,REGULAR,5640802,1910254,2016-04-30 20:00:00,2016-04-30 16:00:00,5640454.0,1910197.0,04:00:00,348.0,57.0,5,20


In [54]:
df_weekday = df[df.WEEKDAY < 5]
df_weekend = df[df.WEEKDAY > 4]

df_weekend.head(5)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,PREV_DATETIME,PREV_ENTRIES,PREV_EXITS,TIME_INTERVAL,ENTRY_COUNT,EXIT_COUNT,WEEKDAY,HOUR
1,A002,R051,02-00-00,59 ST,NQR456,BMT,2016-04-30,1900-01-01 04:00:00,REGULAR,5639991,1909993,2016-04-30 04:00:00,2016-04-30 00:00:00,5639941.0,1909983.0,04:00:00,50.0,10.0,5,4
2,A002,R051,02-00-00,59 ST,NQR456,BMT,2016-04-30,1900-01-01 08:00:00,REGULAR,5640014,1910024,2016-04-30 08:00:00,2016-04-30 04:00:00,5639991.0,1909993.0,04:00:00,23.0,31.0,5,8
3,A002,R051,02-00-00,59 ST,NQR456,BMT,2016-04-30,1900-01-01 12:00:00,REGULAR,5640158,1910134,2016-04-30 12:00:00,2016-04-30 08:00:00,5640014.0,1910024.0,04:00:00,144.0,110.0,5,12
4,A002,R051,02-00-00,59 ST,NQR456,BMT,2016-04-30,1900-01-01 16:00:00,REGULAR,5640454,1910197,2016-04-30 16:00:00,2016-04-30 12:00:00,5640158.0,1910134.0,04:00:00,296.0,63.0,5,16
5,A002,R051,02-00-00,59 ST,NQR456,BMT,2016-04-30,1900-01-01 20:00:00,REGULAR,5640802,1910254,2016-04-30 20:00:00,2016-04-30 16:00:00,5640454.0,1910197.0,04:00:00,348.0,57.0,5,20


In [ ]:

df_weekend.head(5)

In [ ]:
df_weekend = df_weekend[['STATION', 'LINENAME', 'WEEKDAY', 'HOUR', 'ENTRY_COUNT', 'EXIT_COUNT', 'DATE']]
df_weekend.head(5)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

%matplotlib inline
import seaborn as sns

ax = plt.axes()
df_weekend_top_times = df_weekend.groupby('HOUR',as_index=False)['ENTRY_COUNT'].mean()
sns.barplot(x='HOUR', y='ENTRY_COUNT', data=df_weekend_top_times)
ax.set_ylabel('4 hour entry counts')

In [ ]:
df_weekend_daily_entries = df_weekend.groupby(['STATION', 'LINENAME', 'DATE'], as_index=False)['ENTRY_COUNT'].sum()


df_weekend_daily_entries = df_weekend_daily_entries.groupby(['STATION', 'LINENAME'], as_index=False)['ENTRY_COUNT'].mean()
df_weekend_daily_entries.head(10)

In [ ]:
df_weekend_top_stations = df_weekend_daily_entries.sort_values(by=['ENTRY_COUNT']).tail(10)
df_weekend_top_stations

In [ ]:
sns.barplot(x='STATION', y='ENTRY_COUNT', data=df_weekend_top_stations)
ax = plt.axes()
ax.set_title('Top stations on the weekends')
ax.set_ylabel('Daily entries')
for l in ax.get_xticklabels():
    l.set_rotation(90)

In [ ]:
df_weekend_top_times = df_weekend.groupby(['STATION', 'DATE', 'WEEKDAY', 'HOUR'], as_index=False)['ENTRY_COUNT'].sum()

In [ ]:
df_weekend_top_times = df_weekend_top_times.groupby(['STATION', 'WEEKDAY', 'HOUR'], as_index=False)['ENTRY_COUNT'].mean()
df_weekend_top_times = df_weekend_top_times.sort_values(by=['ENTRY_COUNT']).tail(20)
weekend_dict = {5: "Saturday", 6: "Sunday"}
df_weekend_top_times['WEEKDAY'] = df_weekend_top_times['WEEKDAY'].map(weekend_dict)
df_weekend_top_times['DESCRIPTION'] = df_weekend_top_times.STATION + " " \
    + df_weekend_top_times.WEEKDAY + " at " + \
    df_weekend_top_times.HOUR.map(str)
sns.barplot(x='DESCRIPTION', y='ENTRY_COUNT', data=df_weekend_top_times)
ax = plt.axes()
ax.set_title('Top stations on the weekends and its time')
ax.set_ylabel('Daily entries')
for l in ax.get_xticklabels():
    l.set_rotation(90)

In [ ]:
df_herald_sq = df[df.STATION == '34 ST-HERALD SQ'][['DATE', 'TIME', 'ENTRY_COUNT']]
df_herald_sq.head(10)

In [ ]:
table = df_herald_sq.pivot_table('ENTRY_COUNT', 'DATE', 'TIME')
sns.heatmap(table.head(20))